In [77]:
import pandas as pd
import sqlite3

In [78]:
# Load the subset dataset

df = pd.read_csv('Data/creditcard_subset.csv')
print("Columns:", df.columns.tolist())
print("Shape:", df.shape)

Columns: ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class']
Shape: (50000, 31)


In [79]:
# Create SQLite database
conn = sqlite3.connect('paysim.db')
df.to_sql('transactions', conn, if_exists='replace', index=False)
print("Database created and data loaded.")

Database created and data loaded.


Fraud Count by Hour:

In [80]:
query_hour = """
SELECT CAST((Time / 3600) % 24 AS INTEGER) AS Hour, COUNT(*) AS Fraud_Count
FROM transactions
WHERE Class = 1
GROUP BY CAST((Time / 3600) % 24 AS INTEGER)
ORDER BY Hour ASC
"""
fraud_by_hour = pd.read_sql(query_hour, conn)
fraud_by_hour.to_csv('fraud_by_hour.csv', index=False)
print(fraud_by_hour.head())

   Hour  Fraud_Count
0     0            1
1     1            1
2     2            5
3     3            2
4     4            4


Average Amount by Fraud Status:

In [81]:
query_amount = """
SELECT Class, AVG(Amount) AS Avg_Amount
FROM transactions
GROUP BY Class
"""
amount_by_fraud = pd.read_sql(query_amount, conn)
amount_by_fraud.to_csv('amount_by_fraud.csv', index=False)
print(amount_by_fraud)

   Class  Avg_Amount
0      0   87.249921
1      1  164.233855


Proportion of Fraud vs. Non-Fraud

In [82]:
query_pie = """
SELECT Class, COUNT(*) AS Count
FROM transactions
GROUP BY Class
"""
pie_data = pd.read_sql(query_pie, conn)
pie_data.to_csv('pie_data.csv', index=False)
print(pie_data)

   Class  Count
0      0  49917
1      1     83


Top 10 Fraud Transactions by Amount:

In [83]:
query_top = """
SELECT Time, Amount
From transactions
WHERE Class = 1
ORDER BY Amount DESC
LIMIT 10
"""
top_fraud = pd.read_sql(query_top, conn)

# Convert Time to hours
top_fraud['Time_hours'] = top_fraud['Time'] / 3600

# Add a sequential rank for each top fraud case
top_fraud['Case Rank'] = range(1, len(top_fraud) + 1)

# Save to CSV with the new column
top_fraud.to_csv('top_fraud_days.csv', index=False)
print(top_fraud)


       Time   Amount  Time_hours  Case Rank
0  154278.0  1504.93   42.855000          1
1   62467.0  1402.16   17.351944          2
2  154309.0  1096.99   42.863611          3
3  147501.0   996.27   40.972500          4
4   87883.0   829.41   24.411944          5
5   39729.0   776.83   11.035833          6
6   64785.0   720.38   17.995833          7
7  140308.0   592.90   38.974444          8
8   52814.0   519.90   14.670556          9
9   60353.0   454.82   16.764722         10


In [84]:
conn.close()
print("Database connection closed.")

Database connection closed.
